In [ ]:
import tensorflow
import os
import shutil
import keras
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import pandas as pd
import joblib
import tensorflow_addons as tfa
import tempfile
import seaborn as sns

from os import listdir
from os.path import isfile, join


### Defining Model Structure

Here we define our model structure using data augmentation to augment are training dataset and then call this function to create a model.

In [ ]:
image_height, image_width = 500, 500
batch_size = 256
epochs = 3

def create_model(image_height, image_width, threshold):

    # Defining data augmentation layer
    data_augmentation = keras.Sequential([
            keras.layers.RandomFlip("horizontal_and_vertical"),
            keras.layers.RandomRotation(0.2),
            ])
    #Defining model
    model = keras.Sequential([
            data_augmentation,
            keras.layers.Rescaling(1./255),
            keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)),
            keras.layers.MaxPool2D((2, 2)),
            keras.layers.Conv2D(32, (3, 3), activation='relu'),
            keras.layers.MaxPool2D((2, 2)),
            keras.layers.Conv2D(64, (3, 3), activation='relu'),
            keras.layers.MaxPool2D((2, 2)),
            keras.layers.Flatten(),
            keras.layers.Dense(512, activation='relu'),
            keras.layers.Dense(1, activation='sigmoid')
        ])
        
    #Compling model
    model.compile(loss='binary_crossentropy',
                        optimizer='adam', 
                        metrics=[tfa.metrics.F1Score(num_classes = 2, threshold = threshold, average="micro")])
    return model

def plot_result(history):
    acc = history.history['F1Score']
    val_acc = history.history['val_F1Score']
    epochs = range(len(acc))

    plt.plot(epochs, acc, 'b', label='Training accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.figure()

    loss = history.history['loss']
    val_loss = history.history['val_loss']
    plt.plot(epochs, loss, 'b', label='Training Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()

def plot_cm(labels, predictions, threshold):
  cm = confusion_matrix(labels, predictions > threshold)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(threshold))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
  print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
  print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
  print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))

def make_predictions(model, val_ds, threshold):

    "Takes model + validation dataframe as input, returns predictions and labels"

    labels =  np.array([])
    predictions =  np.array([])
    # Iterating through batches and making predictions
    for x, y in val_ds:
        labels = np.concatenate([labels, y.numpy()])
        predictions = np.concatenate([predictions, 
                                        [round(i[0],0) for i in model.predict(x).tolist()]])

    # Converting to 1/0 depending on value relative to threshold
    predictions = [1 if i >= threshold else 0 for i in predictions]
    # Converting to int type
    predictions = np.array(predictions).astype(int)
    labels = np.array(labels).astype(int)

    return predictions, labels
   
   
   

In [ ]:
# Creating model for 500 x 500 image with a 0.5 classification threshold
my_model = create_model(image_height, image_width, 0.5)

### Loading Data
Reaing in the images from our directory

In [ ]:
# Path to read data from
data_dir = './Data-Clean/'

# Defining train an validation splits using 80/20 split
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

### Visualizing Data

In [ ]:
class_names = train_ds.class_names
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

### Training Model

Now that we have our training data and model defined we can now train it

In [ ]:
path = os.getcwd()
file_list = [f for f in listdir(path) if isfile(join(path, f))]
history = ""

# only training if the model doesnt already exist then 
if "my_model.pkl" not in file_list:
  history = my_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
  )
  joblib.dump(my_model, 'my_model.pkl')

### Evaluating Performence

In [ ]:
# First making predictions on test and train datasets
train_predictions_baseline = my_model.predict(train_ds, batch_size=batch_size)
test_predictions_baseline = my_model.predict(val_ds, batch_size=batch_size)

In [ ]:
predictions, labels = make_predictions(my_model, val_ds, 0.5)

In [ ]:
print(len(predictions), len(labels))

In [ ]:
# Generating confusion matrix on test set
plot_cm(labels, predictions, 0.5)

### Retraining Model with Higher Classification Threshold
This will help eliminate false positives and address our imbalanced dataset

In [11]:
low_threshold_history = ""
high_threshold_history = ""
low_threshold_model = create_model(image_height, image_width, 0.2)
high_threshold_model = create_model(image_height, image_width, 0.8)

# only training if the model doesnt already exist then 
if "low_threshold_model.pkl" not in file_list:
  low_threshold_history = low_threshold_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
  )
  joblib.dump(low_threshold_model, 'low_threshold_model.pkl')
if "high_threshold_model.pkl" not in file_list:
  high_threshold_history = high_threshold_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
  )
  joblib.dump(high_threshold_model, 'high_threshold_model.pkl')



 4/63 [>.............................] - ETA: 18:07 - loss: 0.7661 - f1_score: 0.8568

In [ ]:
test_predictions_high_threshold = low_threshold_model.predict(val_ds, batch_size=batch_size)

In [ ]:
low_threshold_predictions, low_threshold_labels = make_predictions(low_threshold_model, val_ds, 0.2)

Now we'll evluate performance of our second model with the 0.9 classification threshold

In [ ]:
# Generating confusion matrix on test set
plot_cm(low_threshold_labels, low_threshold_predictions, 0.2)

In [ ]:
high_threshold_predictions, high_threshold_labels = make_predictions(high_threshold_model, val_ds, 0.8)

In [ ]:
# Generating confusion matrix on test set
plot_cm(high_threshold_labels, high_threshold_predictions, 0.8)